In [1]:
import tensorflow as tf
import sklearn as sk
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import random
import os
import seaborn as sns
import numpy as np
sns.set()

import plotly.io as pio
pio.renderers.default = 'notebook'


In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.10.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
df = pd.read_csv("../Loan_status_prediction-TFX-pipeline/data/loan_data_set.csv")
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [9]:
import pandas as pd

# Assuming df is your DataFrame
df['Credit_History'] = df['Credit_History'].replace("1.0 ", 1)
df['Credit_History'] = df['Credit_History'].replace('0.0' , 0)
# Display the DataFrame
df


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [ ]:
NUMERICAL_FEATURES = ['ApplicantIncome', 'CoapplicantIncome','Loan_Amount_Term']

    CATEGORICAL_FEATURES =['Gender', 'Married', 'Dependents', 'Education',
           'Self_Employed','Property_Area','Loan_Status']

    NUMERICAL_CATEGORICAL_FEATURES=['Credit_History']

    LABEL_KEY = 'LoanAmount'

In [ ]:
def build_pipeline(filepath:str, numerical_features:list=NUMERICAL_FEATURES,
                  categorical_features:list=CATEGORICAL_FEATURES,
                  numerical_categorical_feature=NUMERICAL_CATEGORICAL_FEATURES,
                  label:str=LABEL_KEY):
    
    df = pd.read_csv(filepath)


    COLUMNS_TO_DROP=["Loan_ID"]
    
    df.drop('Loan_ID',axis=True,inplace=True)
    
    for column in CATEGORICAL_FEATURES:
        df[column].fillna(df[column].mode()[0], inplace=True)

    # filling with median for numerical variables
    for column in NUMERICAL_FEATURES:
        tf.keras.layers.CategoryEncoding(
    num_tokens=None, output_mode='multi_hot', sparse=False, **kwargs
)

    for column in NUMERICAL_CATEGORICAL_FEATURES:
        df[column].fillna(df[column].median(), inplace=True)

    # Filling missing values in the label column
    df[LABEL_KEY] = df[LABEL_KEY].fillna(df[LABEL_KEY].median())
    
    df['Dependents'] = df['Dependents'].replace('3+', '3')
    df['Dependents'] = df['Dependents'].astype(int)
    
    df['']
    
    
    le = LabelEncoder()
    for col in df.select_dtypes('object'):
        df[col] = le.fit_transform(df[col])
        
        
    np.random.seed(2)
    msk=np.random.rand(len(df))<0.85

    traindf=df[msk]
    evaldf=df[~msk]

        
    trainds = tf.data.Dataset.from_tensor_slices(dict(traindf)).prefetch(tf.data.AUTOTUNE)
    evalds = tf.data.Dataset.from_tensor_slices(dict(evaldf)).prefetch(tf.data.AUTOTUNE)   
        
    print(traindf.head(5))    
        
    return trainds , evalds

In [ ]:
build_pipeline(filepath="../Loan_status_prediction-TFX-pipeline/data/data.csv")

In [ ]:
np.random.seed(2)
msk=np.random.rand(len(df))<0.85

traindf=df[msk]
evaldf=df[~msk]

traindf.to_csv("../Loan_status_prediction-TFX-pipeline/data/train.csv",index=False, header=False)

evaldf.to_csv("../Loan_status_prediction-TFX-pipeline/data/eval.csv",index=False, header=False)

In [ ]:
def feature_engg(features):
    features['total_income'] = (features['ApplicantIncome']+features['CoapplicantIncome'])
    return features

In [ ]:
CSV_COLUMNS = traindf.columns
LABEL_COLUMNS = 'LoanAmount'
# Default values for each column
DEFAULTS = [['na'], ['Male'], ['Yes'], ['0'], ['Not Graduate'], ['No'], [0.0], [0.0],
            [0.0], [0.0], [0.0], ['Urban'], ['N']]
batch_size=32

            
trainds = tf.data.experimental.make_csv_dataset("../Loan_status_prediction-TFX-pipeline/data/train.csv",
                                                       batch_size=batch_size,
                                                       column_defaults=DEFAULTS,
                                                       column_names=CSV_COLUMNS,
                                                       label_name=LABEL_COLUMNS,
                                                       num_epochs=100,
                                                       num_parallel_reads=30)


evalds= tf.data.experimental.make_csv_dataset("../Loan_status_prediction-TFX-pipeline/data/eval.csv",
                                                       batch_size=batch_size,
                                                       column_defaults=DEFAULTS,
                                                       column_names=CSV_COLUMNS,
                                                       label_name=LABEL_COLUMNS,
                                                       num_epochs=100,
                                                       num_parallel_reads=30)


In [ ]:
trainds.take(1)


In [ ]:
cat_layer = tf.keras.layers.CategoryEncoding(
    num_tokens=4, output_mode='one_hot', sparse=False
)

In [ ]:
norm_layer = tf.keras.layers.Normalization(
    axis=None)

In [ ]:
one_hot_layer = tf.keras.layers.CategoryEncoding(
    num_tokens=3, output_mode='one_hot')


In [ ]:
normalization_layer = tf.keras.layers.Normalization(mean=2.0, variance=1.0)